In [340]:
import numpy as np
import igl
import meshplot as mp

In [341]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [342]:
# pi: cat mesh
# v: vertex connectivity
# n: vertex normals
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [343]:
# find a closest point to "point" in the "points" cloud
# return index in "points"
def find_closest_point(point, points):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argmin(distances)

In [344]:
def constraints(points, normals, ep = 0.8):
    p_value = [] # closest points
    p_location = [] # values corresponding to p_value, indicate whether inside, outside or on surface

    for i, p in enumerate(points):
        n = normals[i] # point's normal

        # Add a constraint f(pi) = di = 0
        p_value.append(p)
        p_location.append(0)

        # Compute p+ and p-
        p_plus  = p + ep * n
        p_minus = p - ep * n

        index_plus  = find_closest_point(p_plus, points)
        index_minus = find_closest_point(p_minus, points)

        # closest pt to p+ and p- should be at least p itself
        # this means distance_closest_+/- should < distance_+/-_to_p 
        closest_p_plus  = points[index_plus]  
        closest_p_minus = points[index_minus]

        distance_closest_plus  = np.linalg.norm(p_plus -closest_p_plus)
        distance_closest_minus = np.linalg.norm(p_minus - closest_p_minus)

        distance_plus_to_p = np.linalg.norm(p_plus - p)
        distance_minus_to_p = np.linalg.norm(p_minus - p)

        ep_plus = ep 
        while distance_closest_plus > distance_plus_to_p:
            ep_plus /= 2
            p_plus = p + ep_plus * n
            index_plus  = find_closest_point(p_plus, points)
            closest_p_plus  = points[index_plus]
            distance_closest_plus  = np.linalg.norm(p_plus - closest_p_plus)
            distance_plus_to_p = np.linalg.norm(p_plus - p)
        
        ep_minus = ep 
        while distance_closest_minus > distance_minus_to_p:
            ep_minus /= 2
            p_minus = p - ep_minus * n
            index_minus  = find_closest_point(p_minus, points)
            closest_p_minus  = points[index_minus]
            distance_closest_minus  = np.linalg.norm(p_minus - closest_p_minus)
            distance_minus_to_p = np.linalg.norm(p_minus - p)

        p_value.append(p_plus)
        p_value.append(p_minus)
        p_location.append(ep_plus)
        p_location.append(-ep_minus)

        # print(p, ": ", p_plus, ep_plus, p_minus, -ep_minus)
    return np.array(p_value), np.array(p_location)

p_value, p_location = constraints(pi, ni)


In [345]:
def plot_colors(p_value, p_location):
    colors = np.zeros((len(p_location), 3)) 

    for i, l in enumerate(p_location):
        if l == 0:
            colors[i] = [0, 0, 1]  #blue, on
        elif l > 0:
            colors[i] = [1, 0, 0]  #red, outside
        else:
            colors[i] = [0, 1, 0]  #green, inside

    mp.plot(p_value, shading={"point_size": 4.5}, c=colors)

plot_colors(p_value, p_location)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0067558…

# MLS function

In [346]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [347]:
# Generate grid n x n x n
x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius

In [348]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [349]:
def wendland_weight(r, wendlandRadius):
    # r = r / wendlandRadius # normalized r
    # r = np.maximum(0, 1 - r / wendlandRadius)
    if r >= 1:
        return 0
    return (1 - r) ** 4 * (4 * r + 1) 

def closest_points(point, points, h):
    indices = []
    distances = np.linalg.norm(points - point, axis=1)
    # print(distances)
    for i, d in enumerate(distances):
        if d < h:
            indices.append(i)
    return indices

def mls_interpolation(x, p_value, p_location, wendlandRadius, polyDegree):
    coefficients = (polyDegree + 1) * (polyDegree + 2) * (polyDegree + 3) // 6

    fx = [] # initialize grid point xi values

    for i, xi in enumerate(x):
        indices = closest_points(xi, p_value, wendlandRadius)
        # print(indices)
        if len(indices) < 2*coefficients:
            fx.append(1.0)
            continue
        # print(indices)
        constraints_p = p_value[indices]
        constraints_f = p_location[indices]
        
        W = []
        distances = np.linalg.norm(constraints_p - xi, axis=1)
        for d in distances:
            W.append(wendland_weight(d, wendlandRadius))
        W = np.array(W)

        # polynomial basis matrix
        if polyDegree == 0:
            A = np.ones((len(constraints_p), 1))
        elif polyDegree == 1:
            A = np.c_[np.ones(len(constraints_p)), constraints_p - xi]  
        elif polyDegree == 2:
            x_diff = constraints_p - xi
            A = np.c_[np.ones(len(constraints_p)), x_diff, x_diff**2, x_diff[:, 0]*x_diff[:, 1], x_diff[:, 0]*x_diff[:, 2], x_diff[:, 1]*x_diff[:, 2]]
        
        try:
            W = np.diag(W)
            coeffs = np.linalg.solve(A.T @ W @ A, A.T @ W @ constraints_f)

            if polyDegree == 0:
                fx.append(np.dot(coeffs, [1]))
            elif polyDegree == 1:
                fx.append(np.dot(coeffs, np.array([1, xi[0], xi[1], xi[2]])))
                # print(fx[i])
            elif polyDegree == 2:
                fx.append(np.dot(coeffs, np.array([1, xi[0], xi[1], xi[2], xi[0]**2, xi[0]*xi[1], xi[0]*xi[2], xi[1]**2, xi[1]*xi[2], xi[2]**2])))
        
        except np.linalg.LinAlgError:
            fx.append(1.0)
        # print(fx[i])
    
    fx = np.array(fx)
    # print(fx)
    colors = np.zeros((len(x), 3))
    
    for i, f in enumerate(fx):
        if f >= 0:
            colors[i] = [1, 0, 0]  #red, outside
        elif f < 0:
            colors[i] = [0, 1, 0]  #green, inside
     
    return fx, colors   

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

p_value, p_location = constraints(pi, ni)
# print(p_location)
wendlandRadius = 30
polyDegree = 1

fx, colors = mls_interpolation(x, p_value, p_location, wendlandRadius, polyDegree)
# print(fx)
mp.plot(x, c=colors, shading={"point_size": 0.2, "width": 1000, "height": 1000})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
luigi, ni = igl.read_triangle_mesh('data/luigi.off')
ni = igl.per_vertex_normals(luigi, v)
# mp.plot(luigi, shading={"point_size": 10})

def align_GridToPCA(points):
    centroid = np.mean(points, axis=0)
    centered = points - centroid

    covariance = np.cov(centered, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(covariance)
    sorted_vecs = np.argsort(-eigvals)
    eigvecs = eigvecs[:, sorted_vecs]
    
    aligned_points = centered @ eigvecs
    return aligned_points, eigvecs, centroid

def pca_grid(points, n):
    min_corner = np.min(points, axis=0)
    max_corner = np.max(points, axis=0)
    
    x = np.linspace(min_corner[0], max_corner[0], n)
    y = np.linspace(min_corner[1], max_corner[1], n)
    z = np.linspace(min_corner[2], max_corner[2], n)

    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")

    grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

    return grid_points


aligned_points, eigvecs, centroid = align_GridToPCA(luigi)
grid_pca = pca_grid(aligned_points, n= 20)
grid = (grid_pca @ eigvecs.T) + centroid

mp.plot(aligned_points, shading={"point_size": 10})
# mp.plot(grid_pca, shading={"point_size": 5})
# mp.plot(grid, shading={"point_size": 5})

p_value, p_location = constraints(luigi, ni) 

wendlandRadius = 30
polyDegree = 1
ffx, fcolors = mls_interpolation(grid, p_value, p_location, wendlandRadius, polyDegree)

mp.plot(grid, c=fcolors, shading={"point_size": 5})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.677721…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(14.496055…

# Marching to extract surface

In [351]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.003235…